In [ ]:
from utilities import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import arviz as az
import pymc as pm
import aesara.tensor as at
from create_training_instance_dictionary_with_covariates import *
from feature_extraction import *
from sample_from_full_model import *
# Initialize random number generator
RANDOM_SEED = 42
rng = np.random.default_rng(RANDOM_SEED)
print(f"Running on PyMC v{pm.__version__}")

In [ ]:
# Generate simulated data with interactions 
true_sigma_obs = 0.1
N_patients = 100

# True parameter values
P = 3 # Number of covariates
P0 = int(P / 2) # A guess of the true number of nonzero parameters is needed for defining the global shrinkage parameter
X_mean = np.repeat(0,P)
X_std = np.repeat(0.5,P)
#X = np.random.normal(X_mean, X_std, size=(N_patients,P))
X = np.random.uniform(-1, 1, size=(N_patients,P))
X = pd.DataFrame(X, columns = ["Covariate "+str(ii+1) for ii in range(P)])
# These are the true parameters for x1 = 0 (median):
rho_s_population = -0.005
rho_r_population = 0.001
pi_r_population = 0.4
psi_population = 50
theta_rho_s_population_for_x_equal_to_zero = np.log(-rho_s_population)
theta_rho_r_population_for_x_equal_to_zero = np.log(rho_r_population)
theta_pi_r_population_for_x_equal_to_zero  = np.log(pi_r_population/(1-pi_r_population))

true_omega = np.array([0.05, 0.10, 0.15])
true_alpha = np.array([theta_rho_s_population_for_x_equal_to_zero, theta_rho_r_population_for_x_equal_to_zero, theta_pi_r_population_for_x_equal_to_zero])
true_beta_rho_s = np.zeros(P)
true_beta_rho_s[0] = 0.8
true_beta_rho_s[1] = 0
interaction_beta_x1_x2_rho_s = 0 ####################
true_beta_rho_r = np.zeros(P)
true_beta_rho_r[0] = 0.7
true_beta_rho_r[1] = 1.0
true_beta_pi_r = np.zeros(P)
true_beta_pi_r[0] = 0.0
true_beta_pi_r[1] = 1.1

print("true_alpha[0]:", true_alpha[0])
print("true_alpha[1]:", true_alpha[1])
print("true_alpha[2]:", true_alpha[2])
print("true_beta_rho_s: ", true_beta_rho_s)
print("true_beta_rho_r: ", true_beta_rho_r)
print("true_beta_pi_r: ", true_beta_pi_r)

# This could be a function: 
#   X, patient_dictionary = generate_simulated_patients(M_number_of_measurements, days_between_measurements, measurement_times, treatment_history, interaction_beta_x1_x2_rho_s)

M_number_of_measurements = 5
days_between_measurements = int(1500/M_number_of_measurements)
measurement_times = days_between_measurements * np.linspace(0, M_number_of_measurements-1, M_number_of_measurements)
treatment_history = np.array([Treatment(start=0, end=measurement_times[-1], id=1)])

expected_theta_1 = np.reshape(true_alpha[0] + np.dot(X, true_beta_rho_s), (N_patients,1))
print((true_alpha[0] + np.dot(X, true_beta_rho_s)).shape)
print((interaction_beta_x1_x2_rho_s*X["Covariate 1"]*(X["Covariate 2"].T)).shape)
print(expected_theta_1.shape)
expected_theta_1 = np.reshape(true_alpha[0] + np.dot(X, true_beta_rho_s) + np.ravel(interaction_beta_x1_x2_rho_s*X["Covariate 1"]*(X["Covariate 2"].T)), (N_patients,1))
print(expected_theta_1.shape)
expected_theta_2 = np.reshape(true_alpha[1] + np.dot(X, true_beta_rho_r), (N_patients,1))
expected_theta_3 = np.reshape(true_alpha[2] + np.dot(X, true_beta_pi_r), (N_patients,1))

true_theta_rho_s = np.random.normal(expected_theta_1, true_omega[0])
true_theta_rho_r = np.random.normal(expected_theta_2, true_omega[1])
true_theta_pi_r  = np.random.normal(expected_theta_3, true_omega[2])

true_omega_for_psi = 0.1
true_theta_psi = np.random.normal(np.log(psi_population), true_omega_for_psi, size=N_patients)
true_rho_s = - np.exp(true_theta_rho_s)
true_rho_r = np.exp(true_theta_rho_r)
true_pi_r  = 1/(1+np.exp(-true_theta_pi_r))
true_psi = np.exp(true_theta_psi)

patient_dictionary = {}
for training_instance_id in range(N_patients):
    psi_patient_i   = true_psi[training_instance_id]
    pi_r_patient_i  = true_pi_r[training_instance_id]
    rho_r_patient_i = true_rho_r[training_instance_id]
    rho_s_patient_i = true_rho_s[training_instance_id]
    these_parameters = Parameters(Y_0=psi_patient_i, pi_r=pi_r_patient_i, g_r=rho_r_patient_i, g_s=rho_s_patient_i, k_1=0, sigma=true_sigma_obs)
    this_patient = Patient(these_parameters, measurement_times, treatment_history, name=str(training_instance_id))
    patient_dictionary[training_instance_id] = this_patient
    #plot_true_mprotein_with_observations_and_treatments_and_estimate(these_parameters, this_patient, estimated_parameters=[], PLOT_ESTIMATES=False, plot_title=str(training_instance_id), savename="./plots/Bayes_simulated_data/"+str(training_instance_id))

In [ ]:
# Visualize parameter dependancy on covariates 
VISZ = True
if VISZ:
    color_array = X["Covariate 2"].to_numpy()

    fig, ax = plt.subplots()
    ax.set_title("expected_theta_1 depends on covariates 1 and 2")
    points = ax.scatter(X["Covariate 1"], expected_theta_1, c=color_array, cmap="plasma")
    ax.set_xlabel("covariate 1")
    ax.set_ylabel("expected_theta_1")
    cbar = fig.colorbar(points)
    cbar.set_label('covariate 2', rotation=90)
    plt.show()

    fig, ax = plt.subplots()
    ax.set_title("true_theta_rho_s depends on covariates 1 and 2")
    points = ax.scatter(X["Covariate 1"], true_theta_rho_s, c=color_array, cmap="plasma")
    ax.set_xlabel("covariate 1")
    ax.set_ylabel("true_theta_rho_s")
    cbar = fig.colorbar(points)
    cbar.set_label('covariate 2', rotation=90)
    plt.show()

    fig, ax = plt.subplots()
    ax.set_title("true_rho_s depends on covariates 1 and 2")
    points = ax.scatter(X["Covariate 1"], true_rho_s, c=color_array, cmap="plasma")
    ax.set_xlabel("covariate 1")
    ax.set_ylabel("true_rho_s")
    cbar = fig.colorbar(points)
    cbar.set_label('covariate 2', rotation=90)
    plt.show()

In [ ]:
# Transpose and check that dimensions are correct
Y = np.transpose(np.array([patient.Mprotein_values for _, patient in patient_dictionary.items()]))
t = np.transpose(np.array([patient.measurement_times for _, patient in patient_dictionary.items()]))
yi0 = np.array([patient.Mprotein_values[0] for _, patient in patient_dictionary.items()])
yi0 = np.maximum(yi0, 1e-5)
#print("Y:\n", Y)
#print("t:\n", t)
#print("yi0:\n", yi0)
#print("X:\n", X)
X_not_transformed = X.copy().T
print(X_not_transformed.columns.values)
#X = X.T
print("Shapes:")
print("Y:", Y.shape)
print("t:", t.shape)
print("yi0:", yi0.shape)
print("X:", X.shape)
print("Done generating data")

assert X.shape[0] == len(patient_dictionary)
N_cases, P = X.shape
print("N_cases:", N_cases)
print(patient_dictionary[0].treatment_history[-1].end)

In [ ]:
# Check that histories stop at the last M protein measurement
print(patient_dictionary[0].treatment_history[-1].end)
print(patient_dictionary[0].measurement_times[-1])

In [ ]:
# Sample from full model
psi_prior="normal"
N_samples = 1000
N_tuning = 3000
target_accept = 0.99    
max_treedepth = 10
FUNNEL_REPARAMETRIZATION = False
name = "simdata_linear_M_"+str(M_number_of_measurements)+"_P_"+str(P)+"_N_cases_"+str(N_cases)+"_psi_prior_"+psi_prior+"_N_samples_"+str(N_samples)+"_N_tuning_"+str(N_tuning)+"_target_accept_"+str(target_accept)+"_max_treedepth_"+str(max_treedepth)+"_FUNNEL_REPARAMETRIZATION_"+str(FUNNEL_REPARAMETRIZATION)
print("Running"+name)
idata = sample_from_full_model(X, patient_dictionary, name, N_samples=N_samples, N_tuning=N_tuning, target_accept=target_accept, max_treedepth=max_treedepth, psi_prior=psi_prior, FUNNEL_REPARAMETRIZATION=FUNNEL_REPARAMETRIZATION)
# This is an xArray: https://docs.xarray.dev/en/v2022.11.0/user-guide/data-structures.html
print("Done sampling")

In [ ]:
# Convergence checks
def quasi_geweke_test(idata, first=0.1, last=0.5, intervals=20):
    print("Running Geweke test...")
    convergence_flag = True
    for var_name in ['alpha', 'beta_rho_s', 'beta_rho_r', 'beta_pi_r', 'omega', 'theta_rho_s', 'theta_rho_r', 'theta_pi_r', 'rho_s', 'rho_r', 'pi_r']:
        sample_shape = idata.posterior[var_name].shape
        n_chains = sample_shape[0]
        n_samples = sample_shape[1]
        var_dims = sample_shape[2]
        for chain in range(n_chains):
            for dim in range(var_dims):
                all_samples = np.ravel(idata.posterior[var_name][chain,:,dim])
                first_part = all_samples[0:int(n_samples*first)]
                last_part = all_samples[n_samples-int(n_samples*last):n_samples]
                z_score = (np.mean(first_part)-np.mean(last_part)) / np.sqrt(np.var(first_part)+np.var(last_part))
                if abs(z_score) >= 1.960:
                    convergence_flag = False
                    print("Seems like chain",chain,"has not converged in",var_name,"dimension",dim,": z_score is",z_score)
    for var_name in ['sigma_obs']:
        all_samples = np.ravel(idata.posterior[var_name])
        n_samples = len(all_samples)
        first_part = all_samples[0:int(n_samples*first)]
        last_part = all_samples[n_samples-int(n_samples*last):n_samples]
        z_score = (np.mean(first_part)-np.mean(last_part)) / np.sqrt(np.var(first_part)+np.var(last_part))
        if abs(z_score) >= 1.960:
            convergence_flag = False
            print("Seems like chain",chain,"has not converged in",var_name,"dimension",dim,": z_score is",z_score)
    if convergence_flag:
        print("All chains seem to have converged.")
    return 0

quasi_geweke_test(idata, first=0.1, last=0.5)

In [ ]:
# Autocorrelation plots: 
az.plot_autocorr(idata, var_names=["sigma_obs"]);
az.plot_autocorr(idata, var_names=["alpha"]);
az.plot_autocorr(idata, var_names=["beta_rho_s"]);
az.plot_autocorr(idata, var_names=["beta_rho_r"]);
az.plot_autocorr(idata, var_names=["beta_pi_r"]);
az.plot_autocorr(idata, var_names=["omega"]);
az.plot_autocorr(idata, var_names=["sigma_obs"]);

In [ ]:
# Posterior means 
print("List of covariates:", idata.posterior.coords['predictors'])
print('Posterior average of alpha:\n', np.mean(idata.posterior['alpha'], axis=(0,1)))
print('\nPosterior average of beta_rho_s:\n', np.mean(idata.posterior['beta_rho_s'], axis=(0,1)))
print('\nPosterior average of beta_rho_r:\n', np.mean(idata.posterior['beta_rho_r'], axis=(0,1)))
print('\nPosterior average of beta_pi_r:\n', np.mean(idata.posterior['beta_pi_r'], axis=(0,1)))
print('\nPosterior average of omega:\n', np.mean(idata.posterior['omega'], axis=(0,1)))
print('\nPosterior average of sigma_obs:\n', np.mean(idata.posterior['sigma_obs'], axis=(0,1)))

In [ ]:
az.plot_trace(idata, var_names=('alpha', 'beta_rho_s', 'beta_rho_r', 'beta_pi_r', 'omega', 'sigma_obs'), combined=True)
plt.savefig("./plots/posterior_plots/"+name+"-plot_posterior_group_parameters.png")

In [ ]:
# Combined means combine the chains into one posterior. Compact means split into different subplots
az.plot_trace(idata, var_names=('beta_rho_s'), lines=[('beta_rho_s', {}, [0])], combined=False, compact=False)
plt.savefig("./plots/posterior_plots/"+name+"-plot_posterior_uncompact_beta_rho_s.png")
# There seems to be an effect where Age contributes towards higher growth rate for sensitive cells

In [ ]:
# Combined means combine the chains into one posterior. Compact means split into different subplots
az.plot_trace(idata, var_names=('beta_rho_r'), lines=[('beta_rho_r', {}, [0])], combined=False, compact=False)
plt.savefig("./plots/posterior_plots/"+name+"-plot_posterior_uncompact_beta_rho_r.png")

In [ ]:
# Combined means combine the chains into one posterior. Compact means split into different subplots
az.plot_trace(idata, var_names=('beta_pi_r'), lines=[('beta_pi_r', {}, [0])], combined=False, compact=False)
plt.savefig("./plots/posterior_plots/"+name+"-plot_posterior_uncompact_beta_pi_r.png")
# There is also an effect where there are less resistant cells for the older patients.

In [ ]:
if psi_prior=="lognormal":
    az.plot_trace(idata, var_names=('xi'), combined=True)
    plt.savefig("./plots/posterior_plots/"+name+"-plot_posterior_group_parameters_xi.png")
    plt.close()
az.plot_trace(idata, var_names=('theta_rho_s', 'theta_rho_r', 'theta_pi_r', 'rho_s', 'rho_r', 'pi_r'), combined=True)
plt.savefig("./plots/posterior_plots/"+name+"-plot_posterior_individual_parameters.png")
plt.close()
# Test of exploration 
az.plot_energy(idata)
plt.savefig("./plots/posterior_plots/"+name+"-plot_energy.png")
plt.close()
# Plot of coefficients
az.plot_forest(idata, var_names=["alpha"], combined=True, hdi_prob=0.95, r_hat=True)
plt.savefig("./plots/posterior_plots/"+name+"-plot_forest_alpha.png")
az.plot_forest(idata, var_names=["beta_rho_s"], combined=True, hdi_prob=0.95, r_hat=True, rope=(0,0))
plt.savefig("./plots/posterior_plots/"+name+"-plot_forest_beta_rho_s.png")
plt.close()
az.plot_forest(idata, var_names=["beta_rho_r"], combined=True, hdi_prob=0.95, r_hat=True, rope=(0,0))
plt.savefig("./plots/posterior_plots/"+name+"-plot_forest_beta_rho_r.png")
plt.close()
az.plot_forest(idata, var_names=["beta_pi_r"], combined=True, hdi_prob=0.95, r_hat=True, rope=(0,0))
plt.savefig("./plots/posterior_plots/"+name+"-plot_forest_beta_pi_r.png")
plt.close()
az.plot_forest(idata, var_names=["theta_rho_s"], combined=True, hdi_prob=0.95, r_hat=True)
plt.savefig("./plots/posterior_plots/"+name+"-plot_forest_theta_rho_s.png")
plt.close()
az.plot_forest(idata, var_names=["theta_rho_r"], combined=True, hdi_prob=0.95, r_hat=True)
plt.savefig("./plots/posterior_plots/"+name+"-plot_forest_theta_rho_r.png")
plt.close()
az.plot_forest(idata, var_names=["theta_pi_r"], combined=True, hdi_prob=0.95, r_hat=True)
plt.savefig("./plots/posterior_plots/"+name+"-plot_forest_theta_pi_r.png")
plt.close()

In [ ]:
# Posterior distribution of y values
sample_shape = idata.posterior['psi'].shape # [chain, n_samples, dim]
n_chains = sample_shape[0]
n_samples = sample_shape[1]
var_dimensions = sample_shape[2] # one per patient
y_resolution = 80 # 1000 crashed the program
# Gather all the parameter estimates and get y values for each parameter set
posterior_parameters = np.empty(shape=sample_shape, dtype=object)
predicted_y_values = np.empty((n_chains, n_samples, N_patients, y_resolution)) # np.empty((sample_shape+(y_resolution,)))
predicted_y_resistant_values = np.empty_like(predicted_y_values)

def get_predicted_y_values(iterable_item):
    ii,ch,sa = iterable_item
    patient = patient_dictionary[ii]
    measurement_times = patient.get_measurement_times()
    treatment_history = patient.get_treatment_history()
    first_time = min(measurement_times[0], treatment_history[0].start)
    plotting_times = np.linspace(first_time, int(measurement_times[-1]), y_resolution) #int((measurement_times[-1]+1)*10))
    this_sigma_obs = np.ravel(idata.posterior['sigma_obs'][ch,sa])
    this_psi       = np.ravel(idata.posterior['psi'][ch,sa,ii])
    this_pi_r      = np.ravel(idata.posterior['pi_r'][ch,sa,ii])
    this_rho_s     = np.ravel(idata.posterior['rho_s'][ch,sa,ii])
    this_rho_r     = np.ravel(idata.posterior['rho_r'][ch,sa,ii])
    these_parameters = Parameters(Y_0=this_psi, pi_r=this_pi_r, g_r=this_rho_r, g_s=this_rho_s, k_1=0, sigma=this_sigma_obs)
    resistant_parameters = Parameters((these_parameters.Y_0*these_parameters.pi_r), 1, these_parameters.g_r, these_parameters.g_s, these_parameters.k_1, these_parameters.sigma)
    target_predicted_y_values = measure_Mprotein_noiseless(these_parameters, plotting_times, treatment_history) # Predicted total M protein
    return target_predicted_y_values
    #target_predicted_y_resistant_values = measure_Mprotein_noiseless(resistant_parameters, plotting_times, treatment_history) # Predicted resistant part
    #return (target_predicted_y_values, target_predicted_y_resistant_values)
    ##return (these_parameters, target_predicted_y_values, target_predicted_y_resistant_values)

index_set_iterable = np.array([[[(ii,ch,sa) for sa in range(n_samples)] for ch in range(n_chains)] for ii in range(N_patients)])
index_set_iterable_1d = np.reshape(index_set_iterable, (n_samples * n_chains * N_patients, 3))
with Pool(7) as pool:
    pool_predicted_y_values = pool.map(get_predicted_y_values, index_set_iterable_1d)

In [ ]:
#predicted_y_values_both_res_and_total = np.reshape(pool_predicted_y_values, (n_chains, n_samples, N_patients, y_resolution, 2))
#predicted_y_values = predicted_y_values_both_res_and_total[:,:,:,:,0]
predicted_y_values = np.reshape(pool_predicted_y_values, (n_chains, n_samples, N_patients, y_resolution))
print(predicted_y_values.shape)
print(predicted_y_values[0,0,0,:])

In [ ]:
# Old version ran but crashed the Kernel 
sample_shape = idata.posterior['psi'].shape # [chain, n_samples, dim]
n_chains = sample_shape[0]
n_samples = sample_shape[1]
var_dimensions = sample_shape[2] # one per patient
y_resolution = 80 # 1000 crashed the program
# Gather all the parameter estimates and get y values for each parameter set
posterior_parameters = np.empty(shape=sample_shape, dtype=object)
predicted_y_values = np.empty((n_chains, n_samples, N_patients, y_resolution)) # np.empty((sample_shape+(y_resolution,)))
predicted_y_resistant_values = np.empty_like(predicted_y_values)
for ii in range(N_patients): # per patient
    patient = patient_dictionary[ii]
    measurement_times = patient.get_measurement_times() 
    treatment_history = patient.get_treatment_history()
    first_time = min(measurement_times[0], treatment_history[0].start)
    plotting_times = np.linspace(first_time, int(measurement_times[-1]), y_resolution) #int((measurement_times[-1]+1)*10))
    for ch in range(n_chains):
        for sa in range(n_samples):
            this_sigma_obs = np.ravel(idata.posterior['sigma_obs'][ch,sa])
            this_psi       = np.ravel(idata.posterior['psi'][ch,sa,ii])
            this_pi_r      = np.ravel(idata.posterior['pi_r'][ch,sa,ii])
            this_rho_s     = np.ravel(idata.posterior['rho_s'][ch,sa,ii])
            this_rho_r     = np.ravel(idata.posterior['rho_r'][ch,sa,ii])
            posterior_parameters[ch,sa,ii] = Parameters(Y_0=this_psi, pi_r=this_pi_r, g_r=this_rho_r, g_s=this_rho_s, k_1=0, sigma=this_sigma_obs)
            these_parameters = posterior_parameters[ch,sa,ii]
            resistant_parameters = Parameters((these_parameters.Y_0*these_parameters.pi_r), 1, these_parameters.g_r, these_parameters.g_s, these_parameters.k_1, these_parameters.sigma)
            # Predicted total M protein
            predicted_y_values[ch,sa,ii] = measure_Mprotein_noiseless(these_parameters, plotting_times, treatment_history)
            # Predicted resistant part
            predicted_y_resistant_values[ch,sa,ii] = measure_Mprotein_noiseless(resistant_parameters, plotting_times, treatment_history)

In [ ]:
print(predicted_y_values.shape)
print(predicted_y_values[0,0,0].shape)
print(predicted_y_resistant_values[0,0,0].shape)

# Group measurements by patient and time
flat_pred_y_values = np.reshape(predicted_y_values, (n_chains*n_samples,N_patients,y_resolution))
# sort the arrays of y
sorted_pred_y_values = np.sort(flat_pred_y_values, axis=0)
print(sorted_pred_y_values.shape)

In [ ]:
# Test that we got what we wanted
training_instance_id = 0
for index, confidence_level in enumerate([0.05, 0.25, 0.45]):
    # Get index to find right value 
    lower_index = int(confidence_level*n_chains*n_samples)
    upper_index = int((1-confidence_level)*n_chains*n_samples)
    # index at intervals to get 95 % limit value
    lower_limits = sorted_pred_y_values[lower_index,training_instance_id,:]
    upper_limits = sorted_pred_y_values[upper_index,training_instance_id,:]       #color=color_array[index]
    print(lower_limits)
    print(upper_limits)

In [ ]:
# Plot posterior confidence intervals 
for training_instance_id, patient in patient_dictionary.items():
    savename = "./plots/Bayesian_estimates_simdata_linearmodel/CI_training_id_"+str(training_instance_id)+"_M_"+str(M_number_of_measurements)+"_P_"+str(P)+"_N_cases_"+str(N_cases)+"_psi_prior_"+psi_prior+"_N_samples_"+str(N_samples)+".png"
    plot_posterior_confidence_intervals(training_instance_id, patient, sorted_pred_y_values, parameter_estimates=[], PLOT_POINT_ESTIMATES=False, PLOT_TREATMENTS=False, plot_title="Posterior CI for patient "+str(training_instance_id), savename=savename, y_resolution=y_resolution, n_chains=n_chains, n_samples=n_samples)

In [30]:
print(predicted_y_values[0,0,0])

[ 49.65575266  49.5763995   49.59428073  49.70658323  49.91075698
  50.20450255  50.5857593   51.05269448  51.60369304  52.23734813
  52.95245237  53.74798965  54.62312761  55.57721063  56.60975346
  57.72043523  58.90909408  60.17572216  61.52046114  62.94359804
  64.4455616   66.0269189   67.68837239  69.43075731  71.25503938
  73.16231281  75.15379866  77.23084346  79.39491808  81.64761697
  83.9906575   86.42587976  88.95524638  91.58084279  94.30487756
  97.12968303 100.05771617 103.09155962 106.23392296 109.48764419
 112.85569135 116.34116449 119.94729763 123.67746113 127.53516405
 131.52405691 135.64793445 139.91073871 144.31656226 148.86965164
 153.57441095 158.43540569 163.45736676 168.64519469 174.00396405
 179.53892805 185.25552339 191.15937528 197.2563027  203.55232384
 210.05366184 216.76675064 223.69824117 230.85500769 238.24415447
 245.87302259 253.74919708 261.88051434 270.2750697  278.94122541
 287.8876188  297.12317077 306.65709453 316.49890473 326.65842683
 337.14580

In [ ]:
# Generate test patients. Generate X and true parameters:
N_patients_test = 20
X_test = np.random.uniform(-1, 1, size=(N_patients_test,P))
X_test = pd.DataFrame(X, columns = ["Covariate "+str(ii+1) for ii in range(P)])

expected_theta_1_test = np.reshape(true_alpha[0] + np.dot(X_test, true_beta_rho_s) + np.ravel(interaction_beta_x1_x2_rho_s*X["Covariate 1"]*(X_test["Covariate 2"].T)), (N_patients_test,1))
expected_theta_2_test = np.reshape(true_alpha[1] + np.dot(X_test, true_beta_rho_r), (N_patients_test,1))
expected_theta_3_test = np.reshape(true_alpha[2] + np.dot(X_test, true_beta_pi_r), (N_patients_test,1))

true_theta_rho_s_test = np.random.normal(expected_theta_1_test, true_omega[0])
true_theta_rho_r_test = np.random.normal(expected_theta_2_test, true_omega[1])
true_theta_pi_r_test  = np.random.normal(expected_theta_3_test, true_omega[2])

true_theta_psi_test = np.random.normal(np.log(psi_population), true_omega_for_psi, size=N_patients_test)
true_rho_s_test = - np.exp(true_theta_rho_s_test)
true_rho_r_test = np.exp(true_theta_rho_r_test)
true_pi_r_test  = 1/(1+np.exp(-true_theta_pi_r_test))
true_psi_test = np.exp(true_theta_psi_test)

# Generate test patients dictionary with measurements Y
patient_dictionary_test = {}
for test_id in range(N_patients):
    psi_patient_i   = true_psi_test[test_id]
    pi_r_patient_i  = true_pi_r_test[test_id]
    rho_r_patient_i = true_rho_r_test[test_id]
    rho_s_patient_i = true_rho_s_test[test_id]
    these_parameters = Parameters(Y_0=psi_patient_i, pi_r=pi_r_patient_i, g_r=rho_r_patient_i, g_s=rho_s_patient_i, k_1=0, sigma=true_sigma_obs)
    this_patient = Patient(these_parameters, measurement_times, treatment_history, name=str(test_id))
    patient_dictionary_test[test_id] = this_patient
    #plot_true_mprotein_with_observations_and_treatments_and_estimate(these_parameters, this_patient, estimated_parameters=[], PLOT_ESTIMATES=False, plot_title=str(test_id), savename="./plots/Bayes_simulated_data/"+str(test_id))


In [ ]:
# Predictions for test patients: Use posterior samples of alphas and weights
# Gather all the parameter estimates and get y values for each parameter set
predicted_parameters = np.empty(shape=(n_chains, n_samples, N_patients_test), dtype=object)
predicted_y_values_test = np.empty((n_chains, n_samples, N_patients_test, y_resolution))
predicted_y_resistant_values_test = np.empty_like(predicted_y_values_test)
for ch in range(n_chains):
    for sa in range(n_samples):
        this_sigma_obs = np.ravel(idata.posterior['sigma_obs'][ch,sa])
        this_alpha = np.ravel(idata.posterior['alpha'][ch,sa])
        this_beta_rho_s = np.ravel(idata.posterior['beta_rho_s'][ch,sa])
        this_beta_rho_r = np.ravel(idata.posterior['beta_rho_r'][ch,sa])
        this_beta_pi_r = np.ravel(idata.posterior['beta_pi_r'][ch,sa])

        predicted_theta_1 = np.reshape(this_alpha[0] + np.dot(X, this_beta_rho_s), (N_patients,1))
        predicted_theta_2 = np.reshape(this_alpha[1] + np.dot(X, this_beta_rho_r), (N_patients,1))
        predicted_theta_3 = np.reshape(this_alpha[2] + np.dot(X, this_beta_pi_r), (N_patients,1))

        predicted_rho_s = - np.exp(predicted_theta_1)
        predicted_rho_r = np.exp(predicted_theta_2)
        predicted_pi_r  = 1/(1+np.exp(-predicted_theta_3))

        for ii, patient in patient_dictionary_test.items():
            measurement_times = patient.get_measurement_times()
            treatment_history = patient.get_treatment_history()
            first_time = min(measurement_times[0], treatment_history[0].start)
            plotting_times = np.linspace(first_time, int(measurement_times[-1]), y_resolution) #int((measurement_times[-1]+1)*10))
            this_rho_s     = predicted_rho_s[ii]
            this_rho_r     = predicted_rho_r[ii]
            this_pi_r      = predicted_pi_r[ii]
            this_psi = patient.Mprotein_values[0]
            predicted_parameters[ch,sa,ii] = Parameters(Y_0=this_psi, pi_r=this_pi_r, g_r=this_rho_r, g_s=this_rho_s, k_1=0, sigma=this_sigma_obs)
            these_parameters = predicted_parameters[ch,sa,ii]
            resistant_parameters = Parameters((these_parameters.Y_0*these_parameters.pi_r), 1, these_parameters.g_r, these_parameters.g_s, these_parameters.k_1, these_parameters.sigma)
            # Predicted total M protein
            predicted_y_values_test[ch,sa,ii] = measure_Mprotein_noiseless(these_parameters, plotting_times, treatment_history)            
            # Predicted resistant part
            predicted_y_resistant_values_test[ch,sa,ii] = measure_Mprotein_noiseless(resistant_parameters, plotting_times, treatment_history)
# Group measurements by patient and time
flat_pred_y_values_test = np.reshape(predicted_y_values_test, (n_chains*n_samples,N_patients_test,1000))
# sort the arrays of y
sorted_pred_y_values_test = np.sort(flat_pred_y_values_test, axis=0)

In [ ]:
# Plot posterior predictive confidence intervals 
for test_id, patient in patient_dictionary_test.items():
    savename = "./plots/Bayesian_estimates_simdata_linearmodel/CI_test_id_"+str(test_id)+"_M_"+str(M_number_of_measurements)+"_P_"+str(P)+"_N_cases_"+str(N_cases)+"_psi_prior_"+psi_prior+"_N_samples_"+str(N_samples)+".png"
    plot_posterior_confidence_intervals(test_id, patient, sorted_pred_y_values_test, parameter_estimates=patient.parameters, PLOT_POINT_ESTIMATES=True, PLOT_TREATMENTS=False, plot_title="Posterior predictive CI for patient "+str(test_id), savename=savename, y_resolution=y_resolution, n_chains=n_chains, n_samples=n_samples)